In [2]:
# Let's begin by checking the structure of the uploaded chinook.db SQLite database to explore its tables
import sqlite3

# Path to the uploaded database
db_path = '/Users/manuel/Desktop/DI-Bootcamp/Week5_DataVizualization/D3/ExerciseXP/chinook.db'

# Connect to the database
conn = sqlite3.connect(db_path)

# Create a cursor object
cursor = conn.cursor()

# Query to fetch all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Display the list of tables
tables

[('albums',),
 ('sqlite_sequence',),
 ('artists',),
 ('customers',),
 ('employees',),
 ('genres',),
 ('invoices',),
 ('invoice_items',),
 ('media_types',),
 ('playlists',),
 ('playlist_track',),
 ('tracks',),
 ('sqlite_stat1',)]

In [3]:
# Import necessary modules
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# Create an engine object and connect to the database
engine = create_engine('sqlite:///chinook.db')

# Reflect the database schema
metadata = sqlalchemy.MetaData()
metadata.reflect(engine)

# Automap base
from sqlalchemy.ext.automap import automap_base
Base = automap_base(metadata=metadata)
Base.prepare()

# Create an ORM session
Session = sessionmaker(bind=engine)
session = Session()

# Helper methods for running SQL queries and displaying results using pandas
from IPython.display import display
import pandas as pd

def sql(query):
    print()
    print(query)
    print()

def get_results(query):
    q = query.statement if isinstance(query, sqlalchemy.orm.query.Query) else query
    return pd.read_sql(q, engine)

def display_results(query):
    df = get_results(query)
    display(df)
    sql(query)


In [4]:
# This will print out all the tables in the database
for table_name in metadata.tables.keys():
    print(table_name)


albums
artists
customers
employees
genres
invoice_items
tracks
media_types
invoices
playlist_track
playlists


In [5]:
# Get the Tracks class and query the first three tracks
Tracks = Base.classes.tracks
query = session.query(Tracks).limit(3)
display_results(query)


,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99



SELECT tracks."TrackId" AS "tracks_TrackId", tracks."Name" AS "tracks_Name", tracks."AlbumId" AS "tracks_AlbumId", tracks."MediaTypeId" AS "tracks_MediaTypeId", tracks."GenreId" AS "tracks_GenreId", tracks."Composer" AS "tracks_Composer", tracks."Milliseconds" AS "tracks_Milliseconds", tracks."Bytes" AS "tracks_Bytes", tracks."UnitPrice" AS "tracks_UnitPrice" 
FROM tracks
 LIMIT ? OFFSET ?



In [8]:
# Check columns of the Tracks and Albums tables
print(metadata.tables['tracks'].columns.keys())
print(metadata.tables['albums'].columns.keys())

['TrackId', 'Name', 'AlbumId', 'MediaTypeId', 'GenreId', 'Composer', 'Milliseconds', 'Bytes', 'UnitPrice']
['AlbumId', 'Title', 'ArtistId']


In [9]:
# Get the Tracks and Albums classes
Tracks = Base.classes.tracks
Albums = Base.classes.albums

# Join Tracks and Albums to get the track name and album title
query = session.query(Tracks.Name, Albums.Title).join(Albums, Tracks.AlbumId == Albums.AlbumId).limit(20)
display_results(query)


,Name,Title
0,For Those About To Rock (We Salute You),For Those About To Rock We Salute You
1,Put The Finger On You,For Those About To Rock We Salute You
2,Let's Get It Up,For Those About To Rock We Salute You
3,Inject The Venom,For Those About To Rock We Salute You
4,Snowballed,For Those About To Rock We Salute You
5,Evil Walks,For Those About To Rock We Salute You
6,C.O.D.,For Those About To Rock We Salute You
7,Breaking The Rules,For Those About To Rock We Salute You
8,Night Of The Long Knives,For Those About To Rock We Salute You
9,Spellbound,For Those About To Rock We Salute You



SELECT tracks."Name" AS "tracks_Name", albums."Title" AS "albums_Title" 
FROM tracks JOIN albums ON tracks."AlbumId" = albums."AlbumId"
 LIMIT ? OFFSET ?



In [10]:
# Get the InvoiceItems class
InvoiceItems = Base.classes.invoice_items

# Join the InvoiceItems and Tracks to get the track name and quantity sold for the first 10 sales
query = session.query(Tracks.Name, InvoiceItems.Quantity).join(InvoiceItems, Tracks.TrackId == InvoiceItems.TrackId).limit(10)
display_results(query)


,Name,Quantity
0,Balls to the Wall,1
1,Restless and Wild,1
2,Put The Finger On You,1
3,Inject The Venom,1
4,Evil Walks,1
5,Breaking The Rules,1
6,Dog Eat Dog,1
7,Overdose,1
8,Love In An Elevator,1
9,Janie's Got A Gun,1



SELECT tracks."Name" AS "tracks_Name", invoice_items."Quantity" AS "invoice_items_Quantity" 
FROM tracks JOIN invoice_items ON tracks."TrackId" = invoice_items."TrackId"
 LIMIT ? OFFSET ?



In [11]:
from sqlalchemy import func

# Query the top 10 tracks sold and the number of times they were sold
query = session.query(Tracks.Name, func.sum(InvoiceItems.Quantity).label('total_sales')).\
    join(InvoiceItems, Tracks.TrackId == InvoiceItems.TrackId).group_by(Tracks.Name).\
    order_by(func.sum(InvoiceItems.Quantity).desc()).limit(10)
display_results(query)

,Name,total_sales
0,The Trooper,5
1,Untitled,4
2,The Number Of The Beast,4
3,Sure Know Something,4
4,Hallowed Be Thy Name,4
5,Eruption,4
6,Where Eagles Dare,3
7,Welcome Home (Sanitarium),3
8,Sweetest Thing,3
9,Surrender,3



SELECT tracks."Name" AS "tracks_Name", sum(invoice_items."Quantity") AS total_sales 
FROM tracks JOIN invoice_items ON tracks."TrackId" = invoice_items."TrackId" GROUP BY tracks."Name" ORDER BY sum(invoice_items."Quantity") DESC
 LIMIT ? OFFSET ?



In [ ]:
# Get the Artists class
Artists = Base.classes.artists

# Join Tracks, Albums, and InvoiceItems to get the top 10 selling artists
query = session.query(Artists.Name, func.sum(InvoiceItems.Quantity).label('total_sales')).\
    join(Albums, Artists.ArtistId == Albums.ArtistId).\
    join(Tracks, Albums.AlbumId == Tracks.AlbumId).\
    join(InvoiceItems, Tracks.TrackId == InvoiceItems.TrackId).\
    group_by(Artists.Name).order_by(func.sum(InvoiceItems.Quantity).desc()).limit(10)
display_results(query)
